In [2]:
import keras
keras.__version__

'2.3.1'

In [3]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)


In [5]:
import keras 
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt', 
    origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('Corpus length:', len(text))

print(text[:900])

Corpus length: 600901
preface


supposing that truth is a woman--what then? is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to truth, have been unskilled and unseemly methods for
winning a woman? certainly she has never allowed herself to be won; and
at present every kind of dogma stands with sad and discouraged mien--if,
indeed, it stands at all! for there are scoffers who maintain that it
has fallen, that all dogma lies on the ground--nay more, that it is at
its last gasp. but to speak seriously, there are good grounds for hoping
that all dogmatizing in philosophy, whatever solemn, whatever conclusive
and decided airs it has assumed, may have been only a noble puerilism
and tyronism; and probably the time is at hand when it 


In [51]:
# Next, you’ll extract partially overlapping sequences of length maxlen, one-hot encode
# them, and pack them in a 3D Numpy array x of shape (sequences, maxlen,unique_characters). 
# Simultaneously, you’ll prepare an array y containing the corresponding targets: 
# the one-hot-encoded characters that come after each extracted sequence


# extract sequences of 60 characters
maxlen = 60

# sample a new sequence every three characters
step = 3

# holds the extracted sequences
sentences = []

# holds the targets (the follow-up characters)
next_chars = []


for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Number of sentences:', len(sentences))



# list of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))


# creating Dictionary that maps unique characters to their index in the list “chars”
# always key: value

chars_indices = dict((char, chars.index(char)) for char in chars)

# print(chars_indices["a"]) # the letter a has 27 as a value: 'a': 27
# print(chars_indices)



print('Vectorization...')

# one hot encodes the characters into binary arrays
x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sentences: 200281
Unique characters: 59
Vectorization...


In [53]:
# single layer LSTM model for next-character prediction

from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))



In [54]:
# model compilation configuration

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [55]:
# function to sample the next character given the model's predictions

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [57]:
import random
import sys

for epoch in range(1, 20): #trains the model for 20 (60) epochs
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs = 1) # fits the model for 1 iteration on the data
    
    # selects a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + ' " ')
    
    
    # tries a range of different sampling temperatures
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        # generate 400 characters
        for i in range(400):
            # one hot encodes the characters generated so far
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            # samples the next character
            preds = model.predict(sampled, verbose=0)[0] # what
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
        
        
    

epoch 1
Epoch 1/1
200281/200281 [==============================] - 357s 2ms/step - loss: 1.8709
--- Generating with seed: "ve all in general society,
which says much about men but not " 
------ temperature: 0.2
ve all in general society,
which says much about men but not the sense--in the ward the sensed and the spiritual and the sensed and the sense--and the supply the sensed the was and the suppress and the great the sensed the sensed and the ward and the was the conscience of the semply and the sensed and the same the sensed the man the sense--as the sense-there and the spiritual and the serstand and the sense-toous and the sensed and the sensed and the sensed------ temperature: 0.5
the sense-toous and the sensed and the sensed and the sensed and rausly har any of the senters and the into destivity and atter that the would has sould in the sensed the sempticise and men the sear there the oun of the bearing that has period the inviention of the been the itself--the great the ears and

conable real bamly in "of human dooner, there origin.. but in parabodic nature," right are and nature. thinks whether the moral to dight
immutic more possible to a herogenes immans applear snanuratumes, very
careness. the immorans snations and have acts"" who fain dewerest. a matter of
their anfieand and consected, s------ temperature: 1.2
o fain dewerest. a matter of
their anfieand and consected, sake seem of
liy undoving ipa no greatusoricus aughtroally
still same
lide
of
"kheeness, on theofer can be eour others semined contrary, coonce them things, and a mohtwise, eppose ongenedkel of
anrequine 1ysomethect of
manging, hew legst im pjudvanation to
ramely
there is "or comrence , (lir reabtly
rethers, nof, are
with: "new wotes morals
that immediuan in unmingary oulding of thot ever theirtatiepoch 9
Epoch 1/1
135808/200281 [===================>..........] - ETA: 1:19 - loss: 1.3803

KeyboardInterrupt: 